In [1]:
import sagemaker
from sagemaker.session import TrainingInput
import numpy as np
from sagemaker.serializers import CSVSerializer
from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()

bucket = 'datascience-sagemaker-fernandosousa'
prefix = 'perceptron/op_logicos'

instance_type = 'local' # trocar para um instância, por exemplo ml.m5.large, para treinar e implantar no sagemaker

role = sagemaker.get_execution_role()

In [2]:
# criar dados de treinamento
dados = [
      [0, 0, 0],
      [0, 1, 0],
      [1, 0, 0],
      [1, 1, 1]
    ]
x = np.array(dados)
# save to csv file
np.savetxt('data/train.csv', x, delimiter=',')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

# definir caminhos para sagemaker copiar os dados
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [3]:
estimator = PyTorch(entry_point='perceptron.py',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'error':0.1
                    })

In [4]:
estimator.fit({"training": train_input, "validation": validation_input})

Creating p47qigspg4-algo-1-ygxy5 ... 
Creating p47qigspg4-algo-1-ygxy5 ... done
Attaching to p47qigspg4-algo-1-ygxy5
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,682 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,685 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,697 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,700 sagemaker_pytorch_container.training INFO     Invoking user training script.
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,933 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,950 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
p47qigspg4-algo-1-ygxy5 | 2022-03-02 01:06:06,972 sagemaker-training-toolkit 

In [5]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type, serializer=CSVSerializer())

Attaching to m15eooxqzu-algo-1-5q3qp
m15eooxqzu-algo-1-5q3qp | ['torchserve', '--start', '--model-store', '/.sagemaker/ts/models', '--ts-config', '/etc/sagemaker-ts.properties', '--log-config', '/opt/conda/lib/python3.6/site-packages/sagemaker_pytorch_serving_container/etc/log4j.properties', '--models', 'model.mar']
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:12,402 [INFO ] main org.pytorch.serve.ModelServer - 
m15eooxqzu-algo-1-5q3qp | Torchserve version: 0.3.0
m15eooxqzu-algo-1-5q3qp | TS Home: /opt/conda/lib/python3.6/site-packages
m15eooxqzu-algo-1-5q3qp | Current directory: /
m15eooxqzu-algo-1-5q3qp | Temp directory: /home/model-server/tmp
m15eooxqzu-algo-1-5q3qp | Number of GPUs: 0
m15eooxqzu-algo-1-5q3qp | Number of CPUs: 2
m15eooxqzu-algo-1-5q3qp | Max heap size: 988 M
m15eooxqzu-algo-1-5q3qp | Python executable: /opt/conda/bin/python3.6
m15eooxqzu-algo-1-5q3qp | Config file: /etc/sagemaker-ts.properties
m15eooxqzu-algo-1-5q3qp | Inference address: http://0.0.0.0:8080
m15eooxqzu

In [7]:
response = predictor.predict(np.array([1,1,0]))
response

m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:20,093 [INFO ] W-9001-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 11
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:20,094 [INFO ] W-9001-model_1 ACCESS_LOG - /172.18.0.1:53372 "POST /invocations HTTP/1.1" 200 14
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:20,094 [INFO ] W-9001-model_1 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:ca352d4924c7,timestamp:null
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:20,094 [INFO ] W-9001-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:10.35|#ModelName:model,Level:Model|#hostname:ca352d4924c7,requestID:749881d9-b316-4304-b4ae-118776a4c2d5,timestamp:1646183180
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:20,095 [INFO ] W-9001-model_1 TS_METRICS - QueueTime.ms:0|#Level:Host|#hostname:ca352d4924c7,timestamp:null
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:20,096 [INFO ] W-9001-model_1 TS_METRICS - WorkerThreadTime.ms:4|#Level:Host|#hostname:ca352d4924c7,timestamp:null


array([0.])

m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:59,802 [INFO ] pool-1-thread-3 ACCESS_LOG - /172.18.0.1:56600 "GET /ping HTTP/1.1" 200 0
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:06:59,803 [INFO ] pool-1-thread-3 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:ca352d4924c7,timestamp:null
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:07:13,330 [INFO ] pool-2-thread-1 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:ca352d4924c7,timestamp:1646183233
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:07:13,332 [INFO ] pool-2-thread-1 TS_METRICS - DiskAvailable.Gigabytes:8.038818359375|#Level:Host|#hostname:ca352d4924c7,timestamp:1646183233
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:07:13,332 [INFO ] pool-2-thread-1 TS_METRICS - DiskUsage.Gigabytes:100.0751838684082|#Level:Host|#hostname:ca352d4924c7,timestamp:1646183233
m15eooxqzu-algo-1-5q3qp | 2022-03-02 01:07:13,333 [INFO ] pool-2-thread-1 TS_METRICS - DiskUtilization.Percent:92.6|#Level:Host|#hostname:ca352d4924c7,timestamp:1646183233
m15eoo

In [ ]:
np.array([1,1,0])